## PALM Dash App

v2.0 (2021-3-3)  
James Harvey 

### Purpose
In this notebook we'll be creating a Dash App. The Dash app will be an interactive tool to play around with PALM Library that was developed by Suhas V. 

See PALM repo here: https://github.com/aifimmunology/PALM

### Contents

1. Load essential Libraries 
2. Run App 


### NOTES: 
- still need to do some setting up with packages: <br> 
    <b> NOTE: PALM has a few dependencies I need to install beforehand </b> 
    - install.packages('nloptr') # reminder need to first install this via ubuntu: "sudo apt-get install libnlopt-dev"  
    - install.packages('lme4') 
    - install.packages('ggforce') 
    - install.packages('factoextra')
    - install.packages("/home/jupyter/PALM_Dash/PALM/PALM_0.1.0.tar.gz", repos = NULL, type ="source") # after cloning from repo 
- could potentially utilize long_callbacks when users want to run a particular graph with different parameters 
- TODO: get these in an image, or find some other way to do this set up under the hood 

___
### Essential Library for Dash App 

In [1]:
from jupyter_dash.comms import _send_jupyter_config_comm_request
_send_jupyter_config_comm_request()

<b> additional step: download repo or datasets locally 
    git clone 
    dowonload data 

#### <b> Run App 
    TODO: utilize parse args so users submit: 
        - datamatrix & metadata
        - featureSet 
        - NA_threshold
        - cvThreshold
        - meanThreshold
        - z-cutoff
    
    
The following will be parameters you can use for run_dash.py: 
    - datamatrix 
    - metadata 
    


In [2]:
%run run_dash.py --metadata /home/jupyter/PALM_DASHv2/PALM/data/data_Metadata.Rda \
    --data /home/jupyter/PALM_DASHv2/PALM/data/Olink_NPX_log2_Protein.Rda \
    --datatype bulk \
    --do_outlier 1 \
    --z_cutoff 2 \
    --mean_threshold 1 \
    --cv_threshold 5 \
    --na_threshold 0.4 \
    --feature_set PTID Time

System has not been booted with systemd as init system (PID 1). Can't operate.
Failed to create bus connection: Host is down


/home/jupyter/PALM_DASHv2/PALM/data/Olink_NPX_log2_Protein.Rda
/home/jupyter/PALM_DASHv2/PALM/data/data_Metadata.Rda
Sat Mar 12 00:44:49 2022 : Performing variance decomposition
  |                                                  | 0 % ~calculating  

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


  |+                                                 | 0 % ~01m 12s       |+                                                 | 0 % ~51s           |+                                                 | 0 % ~44s           |+                                                 | 0 % ~41s           |+                                                 | 0 % ~39s           |+                                                 | 1 % ~37s           |+                                                 | 1 % ~36s           |+                                                 | 1 % ~35s           |+                                                 | 1 % ~37s           |+                                                 | 1 % ~36s           |+                                                 | 1 % ~35s           |+                                                 | 1 % ~34s           |+                                                 | 1 % ~33s           |+                                                 | 1 % ~33s 

/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Dash is running on http://127.0.0.1:8050/proxy/8050/



_____
